In [138]:
import os

def processa_pasta(pasta):
    conteudo_total = ''


    for item in os.listdir(pasta):
        caminho_completo = os.path.join(pasta, item)


        if os.path.isfile(caminho_completo) and caminho_completo.endswith('.log'):

            with open(caminho_completo, 'r') as arquivo_log:
                conteudo_total += arquivo_log.read()
        

        elif os.path.isdir(caminho_completo):
            conteudo_total += processa_pasta(caminho_completo)

    return conteudo_total


pasta_inicial = '/home/michel/Documentos/airflow_docker/dags_iventory_v1/logs'


arquivo_saida = '/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n1/concatenado.log'


conteudo_concatenado = processa_pasta(pasta_inicial)


with open(arquivo_saida, 'w') as arquivo_final:
    arquivo_final.write(conteudo_concatenado)


In [139]:
import pandas as pd


file_path = '/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n1/concatenado.log'


data = []
with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        elements = line.strip().split()
        if len(elements) == 7:
            data.append(elements)


df = pd.DataFrame(data, columns=["File Path", "PID", "Runtime", "# DAGs", "# Errors", "Last Runtime", "Last Run"])


result = df[df['File Path'].str.startswith('/opt')]


output_file = '/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n3/output.csv'


result.to_csv(output_file, index=False)


In [140]:
import pandas as pd

df_1 = pd.read_csv('/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n3/output.csv')
df_1

,File Path,PID,Runtime,# DAGs,# Errors,Last Runtime,Last Run
0,/opt/airflow/dags/__pycache__/get_dags_inventa...,603,0.00s,1,0,2.99s,2023-11-23T18:22:00
1,/opt/airflow/dags/__pycache__/get_dags_inventa...,1035,0.01s,1,0,6.72s,2023-11-23T18:30:04
2,/opt/airflow/dags/__pycache__/get_dags_inventa...,240,0.01s,0,1,7.82s,2023-11-23T19:09:14
3,/opt/airflow/dags/__pycache__/get_dags_inventa...,302,0.50s,1,0,1.80s,2023-11-23T19:10:26
4,/opt/airflow/dags/__pycache__/get_dags_inventa...,386,1.05s,1,0,1.24s,2023-11-23T19:11:55
...,...,...,...,...,...,...,...
176,/opt/airflow/dags/__pycache__/get_dags_inventa...,1973,0.01s,0,1,0.69s,2023-11-27T14:02:32
177,/opt/airflow/dags/get_logs_inventary.py,82,1.02s,0,1,30.00s,2023-11-27T16:20:09
178,/opt/airflow/dags/__pycache__/get_dags_inventa...,83,1.01s,0,1,30.00s,2023-11-27T16:20:09
179,/opt/airflow/dags/get_logs_inventary.py,110,0.02s,0,1,1.26s,2023-11-27T16:20:40


In [141]:
df_1["File Path"].unique()

array(['/opt/airflow/dags/__pycache__/get_dags_inventary.py',
       '/opt/airflow/dags/get_logs_inventary.py'], dtype=object)

In [142]:
import pandas as pd
import re

with open('/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n2/dag_processor_manager.log', 'r') as log_file:
    lines = log_file.readlines()

output_rows = []
current_opt_line = None


for line in lines:
    if line.startswith("/opt"):
        current_opt_line = line.strip()  
    elif line.startswith("[") and "] " in line:
        timestamp = line.split('] ')[0][1:] 
        info = line.split('] ')[1].split(' - ')[1].strip()  
        
        if current_opt_line:  
            output_rows.append([current_opt_line, timestamp, info])  
            current_opt_line = None  

# Criando o DataFrame
df = pd.DataFrame(output_rows)

# Renomeando as colunas
df = df.rename(columns={0: 'File Path', 1: 'Timestamp', 2: 'Info'})
df['last_run'] = df['File Path'].apply(lambda x: x.split()[-1] if len(x.split()) > 1 and re.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}', x.split()[-1]) else None)
df['run'] = df['File Path'].apply(lambda x: x.split()[-2] if len(x.split()) > 1 and re.match(r'\d+\.\d+s', x.split()[-2]) else None)
df['erro'] = df['File Path'].apply(lambda x: int(x.split()[-3]) if len(x.split()) > 2 and x.split()[-3].replace('.', '').isdigit() else None)
df['dag'] = df['File Path'].apply(lambda x: x.split()[-4])
df['File Path'] = df['File Path'].apply(lambda x: x.split()[0])

df  


,File Path,Timestamp,Info,last_run,run,erro,dag
0,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:38:33.212+0000,Searching for files in /opt/airflow/dags,None,None,NaN,45
1,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:38:53.341+0000,Searching for files in /opt/airflow/dags,2023-11-23T17:38:13,0.40s,0.0,1
2,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:39:33.662+0000,Searching for files in /opt/airflow/dags,2023-11-23T17:38:44,0.18s,0.0,1
3,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:39:53.837+0000,Searching for files in /opt/airflow/dags,2023-11-23T17:39:14,0.12s,0.0,1
4,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:40:34.060+0000,Searching for files in /opt/airflow/dags,2023-11-23T17:39:44,0.10s,0.0,1
...,...,...,...,...,...,...,...
2311,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:55:53.434+0000,Searching for files in /opt/airflow/dags,2023-11-25T19:55:43,0.60s,1.0,0
2312,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:56:33.757+0000,Searching for files in /opt/airflow/dags,2023-11-25T19:56:15,0.63s,1.0,0
2313,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:56:53.880+0000,Searching for files in /opt/airflow/dags,2023-11-25T19:56:46,0.46s,1.0,0
2314,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:57:34.148+0000,Searching for files in /opt/airflow/dags,2023-11-25T19:57:17,0.74s,1.0,0


In [143]:
import pandas as pd
import re

df_teste = pd.read_csv('/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n3/output2.csv')
df_teste

,File Path,Timestamp,Info
0,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:38:33.212+0000,Searching for files in /opt/airflow/dags
1,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:38:53.341+0000,Searching for files in /opt/airflow/dags
2,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:39:33.662+0000,Searching for files in /opt/airflow/dags
3,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:39:53.837+0000,Searching for files in /opt/airflow/dags
4,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-23T17:40:34.060+0000,Searching for files in /opt/airflow/dags
...,...,...,...
2311,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:55:53.434+0000,Searching for files in /opt/airflow/dags
2312,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:56:33.757+0000,Searching for files in /opt/airflow/dags
2313,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:56:53.880+0000,Searching for files in /opt/airflow/dags
2314,/opt/airflow/dags/__pycache__/get_dags_inventa...,2023-11-25T19:57:34.148+0000,Searching for files in /opt/airflow/dags


In [144]:
df_teste['File Path'][1].split()

['/opt/airflow/dags/__pycache__/get_dags_inventary.py',
 '1',
 '0',
 '0.40s',
 '2023-11-23T17:38:13']

In [145]:
df_teste['File Path'][0].split()

['/opt/airflow/dags/__pycache__/get_dags_inventary.py',
 '45',
 '0.05s',
 '0',
 '0']

## Limpando os dados de get_logs

In [146]:
import csv
import re

def extract_log_info(input_file, output_file):
    with open(input_file, 'r') as log_file:
        lines = log_file.readlines()

    output_rows = []

    for line in lines:
        # Encontrar dados entre colchetes para "date_time"
        date_time_match = re.search(r'\[(.*?)\]', line)
        date_time = date_time_match.group(1) if date_time_match else "***"

        # Encontrar dados entre chaves para "process"
        process_match = re.search(r'\{(.*?)\}', line)
        process = process_match.group(1) if process_match else "***"

        # Encontrar dados após a primeira ocorrência de "INFO - " para "info_1"
        info_1_match = re.search(r'INFO - (.+)', line)
        info_1 = info_1_match.group(1) if info_1_match else "***"

        # Encontrar dados após a segunda ocorrência de "INFO - " para "info_2"
        info_2_match = re.search(r'INFO - (.+?)INFO - (.+)', line)
        info_2 = info_2_match.group(2).strip() if info_2_match else "***"

        output_rows.append([date_time, process, info_1, info_2])

    # Escrever os dados extraídos em um arquivo CSV
    with open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['date_time', 'process', 'info_1', 'info_2'])  # Cabeçalho do CSV
        csv_writer.writerows(output_rows)

# Exemplo de uso:
extract_log_info('/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n2/get_logs_inventary.py.log', '/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n3/arquivo_get_log.csv')


In [151]:
import pandas as pd

df_get = pd.read_csv('/home/michel/Documentos/airflow_docker/dags_iventory_v1/python/n3/arquivo_get_log.csv')
df_get = df_get[df_get['date_time'] != "***"]
df_get = (df_get[df_get['date_time'] != "Errno 13"]).reset_index()
df_get

,index,date_time,process,info_1,info_2
0,0,2023-11-25T15:43:15.472+0000,processor.py:153,Started process (PID=44) to work on /opt/airfl...,***
1,1,2023-11-25T15:43:15.473+0000,processor.py:743,Processing file /opt/airflow/dags/get_logs_inv...,***
2,2,2023-11-25T15:43:15.478+0000,logging_mixin.py:137,[2023-11-25T15:43:15.477+0000] {dagbag.py:538}...,Filling up the DagBag from /opt/airflow/dags/g...
3,3,2023-11-25T15:43:17.495+0000,logging_mixin.py:137,[2023-11-25T15:43:17.493+0000] {dagbag.py:343}...,***
4,12,2023-11-25T15:43:17.495+0000,processor.py:755,***,***
...,...,...,...,...,...
12787,33045,2023-11-25T19:57:55.322+0000,processor.py:743,Processing file /opt/airflow/dags/get_logs_inv...,***
12788,33046,2023-11-25T19:57:55.330+0000,logging_mixin.py:137,[2023-11-25T19:57:55.330+0000] {dagbag.py:538}...,Filling up the DagBag from /opt/airflow/dags/g...
12789,33047,2023-11-25T19:57:55.343+0000,logging_mixin.py:137,[2023-11-25T19:57:55.342+0000] {dagbag.py:343}...,***
12790,33058,2023-11-25T19:57:55.344+0000,processor.py:755,***,***
